# Demo redes Transformers (o LLM) para clasificar (atributo clase discreto), estimar (atributo clase continuo)  o generar (atributo clase texto) usando como entrada Texto
Basado en 

https://keras.io/guides/functional_api/ 

https://keras.io/examples/generative/text_generation_with_miniature_gpt/

https://keras.io/examples/nlp/text_classification_with_transformer/


Nota: la fuente de datos para datos <FilmAffinity.csv> es https://www.kaggle.com/code/ricardomoya/an-lisis-de-sentimientos-clasificaci-n-de-textos/data pero se corrigieron unos detalles que tenía.

In [ ]:
#@title Librerías a usar
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

import tensorflow as tf
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import math

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import ipywidgets as widgets
from ipywidgets import Box, Layout
from IPython.display import clear_output

import unicodedata
import copy
import random
import os
import csv

print("Librerías cargadas")

In [ ]:
#@title Acceder al Drive

# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive')

# directorio local en Google Drive
path = '/content/gdrive/My Drive/IA/demoML/texto/'  #@param {type:"string"}


In [ ]:
#@title Cargar datos

#@markdown ### Archivo de datos a utilizar:
archivo_datos = 'FilmAffinity.csv'  #@param {type:"string"}
#@markdown ### Configuración del archivo CSV:
delimitador_columnas = '\\|\\|' #@param {type:"string"}

## selección de los parámetros 


def checkValidCharacter(str):
  if len(str) == 0:
    return False  
  for ch in str:
    # controla cada caracter
    if (ch != "\n") and \
       (unicodedata.category(ch) in {'Cc', 'Cf', 'Cn', 'Co', 'Cs'}):
      return False
    # nota: ver categorias en https://www.fileformat.info/info/unicode/category/index.htm
  return True    

def limpiarTexto(texto_ori):
  auxList = []
  t = list( texto_ori.strip() )
  for c in t:
    if checkValidCharacter(c):
      auxList.append( c )
  return "".join(auxList)


# función para cargar configuración datos automática
def cargarNombreClases(path, archivo_datos):
  # importa definición de la clase
  arClasesFN = archivo_datos.split('.')[0] + '_nombreClases.txt'
  if os.path.isfile( path + '/' + arClasesFN ):
    with open( path + '/' + arClasesFN, mode='r') as csvfile:
        r = csv.reader(csvfile, delimiter=',')
        auxAtributo = r.__next__()
        auxClases = r.__next__()
    print('\n> Definición de los valores discretos para la clase cargada de ' + arClasesFN +'.\n')
    return auxAtributo[0], ','.join(auxClases)
  else:
    return "", ""


# configura para que muestre todas las columnas y filas
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

# inicializa valores
Xori = None
X = None
Y = None
nombre_atributos_entrada = []
texto_cargado = None

# determina tipo de archivo a usar (si es de Texto o Datos)
if ".csv" in archivo_datos.lower():
    # se condidera un archivo de datos
    esArchivoDatos = True

    # función auxiliara para que no ejecute UI cada vez
    def hacerNada():
      return

    # se define esta función para que se ocupe de aplicar la configuración
    def on_buttonAplicar_clicked(b):  
      print("")
      funcionCambiaSeleccion_ConfigDatos(combo_att_clase.value, texto_nomClases.value, combo_att_entrada.value)

    # aplica configuración de datos
    def funcionCambiaSeleccion_ConfigDatos(attClase, nomClases, att_entrada):
      global Y, Xori, nombre_clases, nombre_atributos_entrada

      if (attClase is None) or (attClase =="") or\
        (att_entrada is None) or (att_entrada ==""):
        return
      
      # si el atributo clase está como de entrada, lo saca (no tiene sentido)
      att_entrada = list(att_entrada)
      if attClase in att_entrada:
        print("Eliminando atributo " + attClase + " como de entrada dado que es clase.")
        att_entrada.remove( attClase )

      if (att_entrada == "") or (len(att_entrada)==0):
        print("No se han definido atributos de entrada!")
        return

      # guarda configuración
      nombre_clases = nomClases
      nombre_atributos_entrada = att_entrada

        # genera los datos solo con los atributos seleccionados
      Y = np.array(df[attClase])
      Xori = np.array(df[att_entrada])
      
      # muestra resultados
      print("\n> Atributos entrada: ", att_entrada)
      print("\t X: ", Xori.shape)
      
      if (nombre_clases is None) or (nombre_clases==""): 
        print("\n> Atributo clase: ", attClase)
      else:
        print("\n> Atributo clase: ", attClase, " [", nombre_clases, "]")
      print("\t Y: ", Y.shape)

    ## aplicación de los parámetros elegidos

    # Carga los datos del CSV y muestra los primeros
    df = pd.read_csv(path + archivo_datos,  sep=delimitador_columnas, engine="python")
    print("Archivo de datos ", archivo_datos, " cargado")

    print("\n> Cabecera: ")
    print(df.head())
    print("\n> Características: ")
    print(df.describe())
    print("\n")

    # intenta cargar configuración asociada a los datos
    # trata de obtener la configuración del archivo asociado
    atributo_clase, nombre_clases = cargarNombreClases(path, archivo_datos)

    # muestra interface para cargar configuración

    # auxiliar para que muestre bien la descripción
    style_3D = {'description_width': 'initial'}

    tit = widgets.Label("Ajuste para configuración de los Datos: ")
        
    # prepara combo para determinar atributo clase
    selecc_atributos = [ ]
    selecc_atributos.extend( df.columns.values.tolist() )
    if (atributo_clase is None) or (atributo_clase=="") or (atributo_clase not in selecc_atributos):
      att_selecc_defecto = 0
    else:
      att_selecc_defecto = selecc_atributos.index(atributo_clase)
    combo_att_clase = widgets.Dropdown(
        options = selecc_atributos,
        value = selecc_atributos[att_selecc_defecto], # mostrar por defecto de config
        description = 'Atributo clase:',
        style=style_3D,
        disabled = False,
    )
    # prepara campo para ingresar nombre clases (toma por defecto de config)
    texto_nomClases = widgets.Text(
        value=nombre_clases,
        placeholder='Ingrese nombre clases (si corresponde) separados por comas',
        description='Nombre clases:',
        style=style_3D,
        disabled=False
    )

    combo_att_entrada = widgets.SelectMultiple(
        options=selecc_atributos,
        value=selecc_atributos,
        #rows=10,
        description='Atributos de entrada:',
        style=style_3D,
        disabled=False
    )

    # prepara botón y grilla con objetos
    btnAplicar = widgets.Button(
        description='Aplicar'
    )
    configDatos_ui = widgets.GridBox(
          children=[tit, combo_att_clase, texto_nomClases, combo_att_entrada, btnAplicar],
          layout=Layout(width='100%')  )
    btnAplicar.on_click(on_buttonAplicar_clicked)

    # clear_output()  
    out_config = widgets.interactive_output(hacerNada, {})  
    display(configDatos_ui)

    # ejecuta para que muestre
    on_buttonAplicar_clicked(btnAplicar)

else:
    # se condidera un archivo de texto
    esArchivoDatos = False
    
    X = None
    Y = None

    # levanta el archivo de texto del Drive para procesar
    texto_cargado = open("".join(path + archivo_datos), 'rb').read().decode(encoding='utf-8', errors='ignore')
    print("Archivo de texto ", archivo_datos, " cargado")

    # saca caracteres en blanco al principio y fin
    # y caracteres de control especiales
    print("> Se limpian caracteres especiales de control")
    texto_cargado = limpiarTexto(texto_cargado)

    print("\n> Características: ")
    print (' -- Tamaño total del texto: {} caracteres'.format(len(texto_cargado)))

    # muestra los primeros 250 caracteres del texto
    print("\n -- Ejemplo: \n", texto_cargado[:250])

In [ ]:
#@title Generar datos de entrada como Texto

incluir_nombre_atributos = False #@param {type:"boolean"}
marcar_separador_atributos = "xATx " #@param {type:"string"}

if esArchivoDatos:
    # para archivo de datos
    if Xori is None:
      raise(Exception("Falta definir configuración de la carga de datos!"))

    print("> Uniendo atributos X en campo de texto de Datos: ")

    print("\n Datos X originales: ", Xori.shape)
    print(Xori[:3])

    X = []
    for xEj in Xori:
      auxX = ""
      for atEj, nomAt in zip(xEj, nombre_atributos_entrada):
        if (auxX != ""):
          auxX = auxX + " " + marcar_separador_atributos
        if incluir_nombre_atributos:
          auxX = auxX + nomAt + " "
        auxX = auxX + str(atEj)
      X.append( limpiarTexto(auxX) )
    X = np.array( X )
    print("\n Nuevo X: ", X.shape)
    print(X[:3])

else:
    # para archivo de texto
    print("No se necesita ejecutar porque se usa archivo de texto.")

In [ ]:
#@title Preparar datos 

#@markdown Determina si el atributo clase debe ser considerado como Discreto o Continuo
considerar_atributo_clase = "dicreto - CLASIFICACION" #@param ["dicreto - CLASIFICACION", "continuo - ESTIMACION", "texto - GENERACION"] 

#@markdown Porcentaje de datos para usar en el entrenamiento:
proporcion_porcentaje_datos_entrenamiento =   75#@param {type:"integer"}

# inicializa
x_train, x_test, y_train, y_test = None, None, None, None

if esArchivoDatos:
    # para archivo de datos

    if X is None or Y is None:
      raise(Exception("Falta definir configuración de la carga de datos!"))

    # determina la proporción a usar para entrenar y probar
    if proporcion_porcentaje_datos_entrenamiento>100:
      propTrain = 1
    elif proporcion_porcentaje_datos_entrenamiento<1:
      propTrain = 0.1
    else:
      propTrain = proporcion_porcentaje_datos_entrenamiento/100

    # determina si es problema de clasificación o estimación o generación de texto
    if (considerar_atributo_clase[0].upper() == "T"):
      # generación de texto
      tipoProblemaAplicar = "T"
    elif (considerar_atributo_clase[0].upper() == "D"):
      # clasificación
      tipoProblemaAplicar = "C"
    else:
      # estimación
      tipoProblemaAplicar = "E"

    # separa al azar usando muestreo con proporción indicada
    if (tipoProblemaAplicar == "C"):
      # intenta hacer muestreo estatificado 
      try:
        x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=(1-propTrain), stratify=Y)
      except ValueError:
        print("-- No se puede aplicar Muestreo Estratificado! -> se usa Muestreo Simple \n")
        # hace muestreo simple
        x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=(1-propTrain))
    else:
      # hace muestreo simple
      x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=(1-propTrain))

    # inicializa
    y_trainEnc = []
    y_testEnc =  []

    CLASES = []
    if (tipoProblemaAplicar == "C"):
      print("> se considera problema de CLASIFICACIÓN \n")

      # define nombre de clases     
      if (nombre_clases == "") or (nombre_clases == "-"):
          # toma los valores de clase orginales del archivo
          print(Y[0])
          if str(Y[0]).replace(".","").isnumeric():
            # Y son numeros
            for val in range(int(np.max(Y))+1):
              CLASES.append( "clase {:>3}".format(val) )
          else:
              # Y no son números          
              CLASES = list(set(Y))
              CLASES.sort()          
              # cambia valores para que sean enteros
              y_train = [ CLASES.index(y) for y in y_train]
              y_test = [ CLASES.index(y) for y in y_test]
      else:
          # toma configuración de nombre de clases
          for val in nombre_clases.split(','):
            CLASES.append( val )

      # genera salida codificada para softMax
      y_trainEnc =  np_utils.to_categorical(y_train)
      y_testEnc =  np_utils.to_categorical(y_test)

      # muestra resultados
      print("> Definición de CLASES: ")
      print(" - dictMapeo (", len(CLASES), "): ", CLASES)

    elif (tipoProblemaAplicar == "T"):
      print("> se considera problema de GENERACIÓN DE TEXTO \n")

    else: # (tipoProblemaAplicar == "E"):
      print("> se considera problema de ESTIMACIÓN \n") 

    print("\n> Para Entrenamiento: ")
    print(" - x_train (cant): ", x_train.shape)
    print(" - y_train (cant): ", len(y_train))
    if (tipoProblemaAplicar == "C"):
      for i in range(len(CLASES)):
        cant = 0
        for y in y_train:
          if i == int(y): cant = cant + 1
        print("    ", CLASES[i], "[", i, "]:", cant)

    print("\n Para Prueba: ")
    print(" - x_test (cant): ", x_test.shape)
    print(" - y_test (cant): ", len(y_test))
    if (tipoProblemaAplicar == "C"):
      for i in range(len(CLASES)):
        cant = 0
        for y in y_test:
          if i == int(y): cant = cant + 1
        print("    ", CLASES[i], "[", i, "]:", cant)

else:
    # para archivo de texto
    tipoProblemaAplicar = "T"
    print("Al usar Archivo de Texto: ")
    print("-- se considera problema de GENERACIÓN DE TEXTO")
    print("-- se utiliza el texto completo para entrenamiento")
    print("-- se divide el texto completo usando saltos de líneas (\\n)")

    # se divide el texto usando salto de líneas
    y_train = texto_cargado.split("\n")
    print("\n> Para Entrenamiento: ")
    print(" - Frases (cant): ", len(y_train))

    if len(y_train)>3:
      print("\nEjemplos: ")
      for i in range(3):
        print("\t", y_train[i])
        

In [ ]:
#@title Definir Encoding & Embeding del Texto

#@markdown Parámetros para Encoding:
tipo_vocabuario = "caracteres" #@param ["palabras", "caracteres"]
tamaño_maximo_vocabulario = 5000 #@param {type:"integer"}
tamaño_maximo_frase_texto = 100 #@param {type:"integer"}
estandarizar_usar_todo_minusculas = True #@param {type:"boolean"}
estandarizar_sacar_puntuacion = True #@param {type:"boolean"}

#@markdown Parámetros para Embeding:
tamaño_dimensiones_vectores = 64 #@param {type:"integer"}

# función auxiliar para codificar texto
def codificarTexto(encoder, texto):
  return encoder(texto).numpy()

def separarTexto(texto , separadorTokens):
  if separadorTokens=="":
    splitText = list(texto)
  else:
    splitText = texto.split(separadorTokens)
  return splitText

# muestra texto codificado
def mostrarCodificarTexto(encoder, texto, separadorTokens=" "):
  textoCodif = codificarTexto(encoder, texto)
  print("Texto: ", texto)
  splitText = separarTexto(texto, separadorTokens)
  print("Vector[" + str(len(splitText)) +  "]: ", splitText)
  print("Codif[" + str(len(textoCodif)) + "]: ", textoCodif)
  return textoCodif

# rellena vector con valores ceros
def rellenar_vector(vec, nshape):
  # hace una copia del vector
  aux = copy.deepcopy(vec)
  # le cambia el tamaño
  aux.resize(nshape, refcheck=False)
  # lo devuelve
  return aux

# aplica configuración
if tamaño_maximo_vocabulario < 10:
  tamaño_maximo_vocabulario = 10

if tamaño_maximo_frase_texto < 10:
  tamaño_maximo_frase_texto = 10

if tamaño_dimensiones_vectores < 10:
  tamaño_dimensiones_vectores = 10

if tipo_vocabuario == "palabras":
  split_type = "whitespace"
  separadorTokens = " "
elif tipo_vocabuario == "caracteres":
  split_type = "character"
  separadorTokens = ""
else:
  # no se usa porque no hace ninguna separación
  split_type = None 
  separadorTokens = ""

if estandarizar_usar_todo_minusculas and estandarizar_sacar_puntuacion:
  standardize_type = "lower_and_strip_punctuation"
elif estandarizar_usar_todo_minusculas:
  standardize_type = "lower"
elif estandarizar_sacar_puntuacion:
  standardize_type = "strip_punctuation"
else:
  standardize_type = None

if esArchivoDatos:
    # para archivo de datos solamente

    # prepara para manejar datos (usados por todos)
    print("\n-- Crear Encoder para datos.")

    # crea codificador
    x_encoderLay = tf.keras.layers.TextVectorization(
        max_tokens = tamaño_maximo_vocabulario,
        standardize = standardize_type,
        split = split_type,
        output_sequence_length = tamaño_maximo_frase_texto,
        name = "encoding-Data")

    # lo inicializa usando los textos de entrada para entrenamiento
    x_encoderLay.adapt(x_train)

    # obtiene el vocabulario
    x_vocab = np.array(x_encoderLay.get_vocabulary())
    ##print( x_vocab[:20] )

    # determina el tamaño del vocabulario
    x_vocab_size = len(x_vocab)
    print("\t tamaño vocabulario de datos = ", x_vocab_size)


if (tipoProblemaAplicar == "T"):

    # prepara para generación de texto (solo datos Y)
    # se usa con archivos de datos o de texto

    # determina tags especiales (constantes) 
    # notar que no se peuden usar símbolos especiales porque luego se filtran
    if tipo_vocabuario == "palabras":
        tag_gentext_begin = "ästartä"
        tag_gentext_end = "üendü"
    else: # tipo_vocabuario == "caracter":
        tag_gentext_begin = "ä"
        tag_gentext_end = "ü"

    # crea codificador
    print("\n-- Crear Encoder para texto.")
    y_encoderLay = tf.keras.layers.TextVectorization(
        max_tokens = tamaño_maximo_vocabulario,
        standardize = standardize_type,
        split = split_type,        
        output_mode = "int",
        output_sequence_length = tamaño_maximo_frase_texto,
        name = "encoding-Text")
    
    # lo inicializa usando los textos de entrada y salida para entrenamiento
    auxYcompleto = [tag_gentext_begin, tag_gentext_end]
    auxYcompleto.extend( y_train )
    if esArchivoDatos:
        # para archivo de datos                
        auxYcompleto.extend( y_test )    
    y_encoderLay.adapt( auxYcompleto )

    # obtiene el vocabulario
    y_vocab = np.array(y_encoderLay.get_vocabulary())
    ##print( y_vocab[:20] )

    # determina el tamaño del vocabulario
    y_vocab_size = len( y_vocab )
    print("\t tamaño vocabulario de texto = ", y_vocab_size)

    # determina codificación de tags
    codif_gentext_begin = y_encoderLay(tag_gentext_begin).numpy()[0]
    codif_gentext_end = y_encoderLay(tag_gentext_end).numpy()[0]
    print("\n-- Tags especiales: ")
    print("\t comienzo = " + tag_gentext_begin + " (" + str(codif_gentext_begin) + ")")
    print("\t fin = " + tag_gentext_end + " (" + str(codif_gentext_end) + ")")

    if esArchivoDatos:
        # para archivo de datos   
        print("\n-- Preparando múltiples entradas y salida para generar texto.")
    else:
        # para archivo de texto   
        print("\n-- Preparando entrada y salida para generar texto.")

    inputs_x_train = [] # entrada para datos
    inputs_prevText_train = [] # entrada para texto previo (o begin)
    y_trainEnc = [] # salida de texto (un elemento por vez)
    for i in range(len(y_train)):
      if esArchivoDatos:
          # determina los datos 
          x = x_train[i]
      # determina texto
      y = y_train[i]
      # si la salida está definida
      if len(y) > 1:
        # codifica la salida (le agrega los tags para marcar comienzo y fin)
        yCodif = codificarTexto(y_encoderLay, " ".join([tag_gentext_begin, y, tag_gentext_end]) )
        yCodifShape = yCodif.shape 
        # va tomando por partes
        if esArchivoDatos:
          # como tiene datos complementarios, puede arrancar con el tag_gentext_begin
          jIni = 1
        else:
          # como no tiene datos complementarios, debe tener además de tag_gentext_begin alguna palabra
          # sino no sabe como arrancar
          jIni = 2
        for j in range(jIni, len(yCodif)):
            if esArchivoDatos:
              # determina entrada de datos (copiada igual por cada una de texto previo)
              inputs_x_train.append( x )
            # determina entrada de texto previo (competa con ceros para tener largo fijo)        
            inputs_prevText_train.append( rellenar_vector(yCodif[:j], yCodifShape) )
            # determina salida texto (un lemento)
            y_trainEnc.append( yCodif[j] )       
            # si llego al tag de END, termina
            if (yCodif[j] == codif_gentext_end) or (yCodif[j] == tag_gentext_end):
              break
        
    # convierte a arrays
    if esArchivoDatos:
        inputs_x_train = np.array( inputs_x_train )
    inputs_prevText_train = np.array( inputs_prevText_train )
    y_trainEnc = np.array( y_trainEnc ) 

    # muestra formas
    if esArchivoDatos:
        print("\tinputs_x_train: ", inputs_x_train.shape)
    print("\tinputs_prevText_train: ", inputs_prevText_train.shape)
    print("\ty_trainEnc: ", y_trainEnc.shape)    

    print("\n> Ejemplos de codificación: \n")
    for _ in range(3):
      posAzar = random.randint(1, len(inputs_prevText_train)) - 1
      if esArchivoDatos:
          print("-INPUT DATOS: ")
          mostrarCodificarTexto(x_encoderLay,  inputs_x_train[posAzar], separadorTokens)
      auxT = ""
      auxV = inputs_prevText_train[posAzar]
      for v in auxV:
        auxT = auxT + separadorTokens + y_vocab[v]
      print("-INPUT TEXTO: ", auxT, " \n--> ",  inputs_prevText_train[posAzar] )        
      auxV = y_trainEnc[posAzar] 
      print("-OUTPUT:      ", y_vocab[auxV], " --> [", auxV, "]")
      print("")

else:
    # muesta ejemplos sólo datos
    print("\n> Ejemplos de codificación: \n")
    for _ in range(3):
      posAzar = random.randint(1, len(x_train)) - 1
      mostrarCodificarTexto(x_encoderLay,  x_train[posAzar], separadorTokens)
      print("")
  

In [ ]:
#@title Establecer modelo Transfomer tipo miniGPT 

#@markdown ### Parámetros para capas Transfotmer:
# cantidad de transformers block
trnsf_cant_bloques = 2 #@param {type: "integer"}
# Hidden layer size in feed forward network inside transformer
trnsf_MultiHeadAttention_head_size = 256 #@param {type: "integer"}
# Number of attention heads
trnsf_MultiHeadAttention_num_heads = 4 #@param {type: "integer"}
trnsf_porc_capa_DropOut = 0.1 #@param {type:"number"}


#@markdown ### Parámetros de las capas Lineales:
lineal_cant_neuronas_capas_ocultas = '128, D,  64' #@param {type:"string"}
lineal_porc_capa_DropOut = 0.4 #@param {type:"number"}

#@markdown ### Parámetros de la capa de Salida:
red_tipo_capa_salida = 'softmax-MultiClase' #@param ["lineal-Numero", "softmax-MultiClase"]

#@markdown ### Parámetros del Optimizador:
opt_tipo = "Adam" #@param ["Gradiente Decreciente", "Adam", "Adadelta", "Adagrad", "Adamax", "Nadam", "FTRL"]
opt_learning_rate = 0.001 #@param {type: "number"}

# aplica parámetros

if trnsf_MultiHeadAttention_head_size < 1:
  trnsf_MultiHeadAttention_head_size = 1

if trnsf_MultiHeadAttention_num_heads < 1:
  trnsf_MultiHeadAttention_num_heads = 1

if trnsf_cant_bloques < 1:
  trnsf_cant_bloques = 1

# chequea configuración de drop out
if trnsf_porc_capa_DropOut <= 0:
  trnsf_porc_capa_DropOut = 0.10
elif trnsf_porc_capa_DropOut > 0.9:
    trnsf_porc_capa_DropOut = 0.9

# cantidad de neuronas ocultas 
hidden_layers = []
for val in lineal_cant_neuronas_capas_ocultas.split(','):
  val = val.strip()
  if val == "D":
    hidden_layers.append( "DropOut" )  
  elif val == "BN":
    hidden_layers.append( "BatchNormalization" )  
  elif val.isnumeric():
    hidden_layers.append( val )
  else:
    print("Capa ", val, "descartada!")

# define si el tipo de capa de salida es softmax( True )  o lineal ( False )
# esto implica también cambiar cómo se codifican los valores de las clases a usar
if (tipoProblemaAplicar == "C"):
  tipo_output_softMax = (red_tipo_capa_salida[:7] == 'softmax')
else:  
  tipo_output_softMax = False

if opt_tipo == "Gradiente Decreciente":
  opt = keras.optimizers.SGD(learning_rate=opt_learning_rate)
elif opt_tipo == "Adam":
  opt = keras.optimizers.Adam(learning_rate=opt_learning_rate)
elif opt_tipo == "Adadelta":
  opt = keras.optimizers.Adadelta(learning_rate=opt_learning_rate)
elif opt_tipo == "Adagrad":
  opt = keras.optimizers.Adagrad(learning_rate=opt_learning_rate)
elif opt_tipo == "Adamax":
  opt = keras.optimizers.Adamax(learning_rate=opt_learning_rate)
elif opt_tipo == "Nadam":
  opt = keras.optimizers.Nadam(learning_rate=opt_learning_rate)
elif opt_tipo == "FTRL":
  opt = keras.optimizers.Ftrl(learning_rate=opt_learning_rate)
else:
  opt = keras.optimizers.Ad

# funciones auxiliares para definir modelo GPT
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    """
    Mask the upper half of the dot product matrix in self attention.
    This prevents flow of information from future tokens to current token.
    1's in the lower triangle, counting from the lower right corner.
    """
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)

class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, dp_rate=0.1, nameCust=None):
        if nameCust is None:
          super().__init__()
        else:
          super().__init__(name=nameCust,)
        self.att = tf.keras.layers.MultiHeadAttention(num_heads, embed_dim)
        self.ffn = keras.Sequential(
            [tf.keras.layers.Dense(feed_forward_dim, activation="relu"), tf.keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(dp_rate)
        self.dropout2 = tf.keras.layers.Dropout(dp_rate)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        attention_output = self.att(inputs, inputs, attention_mask=causal_mask)
        attention_output = self.dropout1(attention_output)
        out1 = self.layernorm1(inputs + attention_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim, nameCust=None):
        super().__init__(trainable=True, name=nameCust, dtype=None, dynamic=False,)
        self.token_emb = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = tf.keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

# variable auxiliar para transformer block
aux_trnsf_tamaño_dimensiones_vectores = 0

# define la arquitectura de capas teniendo en cuenta la definición dada anteriomente
if esArchivoDatos:
  # para archivo de datos

  # agrega capas de entrada y encoding  para datos de entrada  
  x_inputLay = tf.keras.layers.Input(dtype=tf.string, shape=(1,), name="input-Data")
  eachLay = x_encoderLay(x_inputLay)

  # crea capa para embedding
  # que convierte las secuencias de índices codficadas
  # en secuencias de vectores usados para entrenar
  embedding_layer = TokenAndPositionEmbedding(maxlen=tamaño_maximo_frase_texto, 
                                              vocab_size=x_vocab_size, 
                                              embed_dim=tamaño_dimensiones_vectores,
                                              nameCust="Tnsf-embedding-Data")
  eachLay = embedding_layer(eachLay)
  aux_trnsf_tamaño_dimensiones_vectores = aux_trnsf_tamaño_dimensiones_vectores + tamaño_dimensiones_vectores


if (tipoProblemaAplicar == "T"):
  # para archivo de datos o texto
  
  # agrega capas entrada y encoding  para generación de texto  
  ####y_inputLay = tf.keras.layers.Input(dtype=tf.string, shape=(1,), name="input-Text")
  ####eachLay_text = y_encoderLay(y_inputLay)

  y_inputLay = tf.keras.layers.Input(shape=inputs_prevText_train[0].shape, name="input-prevText")
  eachLay_text = y_inputLay

  # crea capa para embedding
  # que convierte las secuencias de índices codficadas
  # en secuencias de vectores usados para entrenar
  embedding_layer = TokenAndPositionEmbedding(maxlen=tamaño_maximo_frase_texto, 
                                              vocab_size=y_vocab_size, 
                                              embed_dim=tamaño_dimensiones_vectores,
                                              nameCust="Tnsf-embedding-prevText")
  eachLay_text = embedding_layer(eachLay_text)
  aux_trnsf_tamaño_dimensiones_vectores = aux_trnsf_tamaño_dimensiones_vectores + tamaño_dimensiones_vectores

  if esArchivoDatos:
    # solo si es archivo de datos
    # une los dos embeddings juntos para ser procesados por la red
    concatLay = tf.keras.layers.concatenate([eachLay, eachLay_text], name="MergeInputs")
    eachLay = concatLay

  else:
      # para archivo de texto  
      eachLay = eachLay_text


# caps de bloques de transformer
for n in range(trnsf_cant_bloques):
    transformer_block = TransformerBlock(embed_dim=aux_trnsf_tamaño_dimensiones_vectores,
                                          num_heads=trnsf_MultiHeadAttention_num_heads,
                                          feed_forward_dim=trnsf_MultiHeadAttention_head_size, 
                                          dp_rate=trnsf_porc_capa_DropOut, 
                                          nameCust="Tnsf-tf_"+str(n+1))
    eachLay = transformer_block(eachLay)

# hace un flatten especial
eachLay = tf.keras.layers.GlobalAveragePooling1D(name="Tnsf-gap")(eachLay)

# agrega capas lineales
auxName = 'lineal_'
auxId = 1 
for val_hid in hidden_layers:  

  if val_hid == "DropOut":
    auxlayerName = "d_"+str(auxId)
    auxId = auxId + 1
    eachLay = tf.keras.layers.Dropout(lineal_porc_capa_DropOut,name=auxlayerName)(eachLay)
  elif val_hid == "BatchNormalization":
    auxlayerName = "bn_"+str(auxId)
    auxId = auxId + 1
    eachLay = tf.keras.layers.BatchNormalization(name=auxlayerName)(eachLay)
  elif val_hid.isnumeric():
    # agrega la capa oculta
    auxlayerName = auxName+str(auxId)
    auxId = auxId + 1
    eachLay = tf.keras.layers.Dense(int(val_hid), name=auxlayerName)(eachLay) # capas ocultas

# agrega capa de salida
if (tipoProblemaAplicar == "T"):
    # se genera capa lineal con salida igual al tamaño del vocabulario
    outputLay = tf.keras.layers.Dense(y_vocab_size, activation='softmax', name='outputText')(eachLay) # capa de salida

elif (tipoProblemaAplicar == "C") and tipo_output_softMax:
    # se genera una capa softmax
    outputLay = tf.keras.layers.Dense(units = len(CLASES), activation='softmax', name='output')(eachLay) # capa de salida
else:
    # se genera una capa lineal con una salida numérica
    outputLay = tf.keras.layers.Dense(1, activation=None, name='output')(eachLay) # capa de salida

# agrega capa de salida
if (tipoProblemaAplicar == "T"):
    # se genera capa lineal con salida igual al tamaño del vocabulario
    outputLay = tf.keras.layers.Dense(y_vocab_size, name='outputText')(eachLay) # capa de salida

elif (tipoProblemaAplicar == "C") and tipo_output_softMax:
    # se genera una capa softmax
    outputLay = tf.keras.layers.Dense(units = len(CLASES), activation='softmax', name='output')(eachLay) # capa de salida
else:
    # se genera una capa lineal con una salida numérica
    outputLay = tf.keras.layers.Dense(1, activation=None, name='output')(eachLay) # capa de salida


# genera el modelo 
if (tipoProblemaAplicar == "T"):
    if esArchivoDatos:
      # para archivo de datos
      # modelo con 2 entradas
      model = keras.Model(
          inputs=[x_inputLay, y_inputLay], 
          outputs=outputLay, 
          name="GenText-GPT")
    
    else:
      # para archivo de texto
      # modelo con entrada de texto previo
      model = keras.Model(
          inputs=y_inputLay, 
          outputs=outputLay, 
          name="GenText-GPT")        
    
    # utiliza un loss especial
    sccLoss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(optimizer=opt, loss=sccLoss, metrics=['accuracy'])

else:
    # modelo normal
    model = keras.Model(
        inputs=x_inputLay,
        outputs=outputLay, 
        name="Text-"+tipoProblemaAplicar+"-GPT")
      
    if  tipo_output_softMax:
        # utiliza un loss de multiple clases
        model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    else:
        # utiliza un loss de valor numérico
        if (tipoProblemaAplicar == "C"):
          model.compile(optimizer=opt, loss='mse', metrics=['accuracy'])
        else:
          model.compile(optimizer=opt, loss='mse', metrics=['RootMeanSquaredError'])

print("Modelo creado con ", len(model.layers), " capas:")
model.summary()
print("\n")
plot_model(model, show_layer_names=True, show_shapes=True)



In [ ]:
#@title Entrenar

cant_epocas_entrenamiento = 15 #@param {type:"integer"}

# cantidad de épocas del entrenamiento
cantEpocas = (1 if cant_epocas_entrenamiento<1 else cant_epocas_entrenamiento)

activar_corte_por_estabilidad_error_val = True #@param {type:"boolean"}


if (tipoProblemaAplicar == "T"):  
    # entrena con múltiples entradas para generar texto
    # (no se usa activar_corte_por_estabilidad_error_val)
    print("\n--- No se utilizan datos de validación ---")
    if esArchivoDatos:
        # para archivo de datos
        print("\n\n>Comienza el Entrenamiento con múltiples entradas:")
        history = model.fit([inputs_x_train, inputs_prevText_train], 
                            y_trainEnc,
                            epochs = cant_epocas_entrenamiento) 
    else:
        # para archivo de texto
        print("\n\n>Comienza el Entrenamiento:")
        history = model.fit(inputs_prevText_train, 
                            y_trainEnc,
                            epochs = cant_epocas_entrenamiento) 

else:
    # entrena normalmente

    # separa al azar usando muestreo al azar del 10%
    # para tomar algunos como datos de validación
    x_t, x_v, y_t, y_v = train_test_split(x_train, 
                                          (y_trainEnc if tipo_output_softMax else y_train), 
                                          test_size=0.1)


    print("\n> De los ", len(x_train), "ejemplos de entrenamiento: ")
    print("            se usan ", len(x_t), "ejemplos para entrenar ")
    print("            y ", len(x_v), "ejemplos para validar.")


    print("\n\n>Comienza el Entrenamiento:")

    if activar_corte_por_estabilidad_error_val:
      # se agrega un callBack para que corte 
      # si el error de validación no sigue bajando
      # y devuelva los mejores pesos obtenidos
      early_stopping_monitor = keras.callbacks.EarlyStopping(
          monitor='val_loss',
          min_delta=0.01,
          patience=20,
          verbose=0,
          mode='min',
          baseline=None,
          restore_best_weights=True
      )
      callbacksEntr = [early_stopping_monitor]
    else:
      early_stopping_monitor = None
      callbacksEntr = []

    # lleva a cabo el entrenamiento
    history = model.fit(x_t, y_t,
              epochs = cantEpocas, 
              validation_data=(x_v, y_v,),
              callbacks=callbacksEntr ) 

print("\n>Entrenamiento Finalizado.")


In [ ]:
#@title Mostrar Gráficos del Entrenamiento
plt.figure(figsize=(15,8)) 
plt.plot(history.history['loss'])
if (tipoProblemaAplicar != "T"):
  plt.plot(history.history['val_loss'])
plt.title('Gráfico del Error del Entrenamiento')
plt.ylabel('')
plt.xlabel('epoch')
if (tipoProblemaAplicar != "T"):
  plt.legend(['entrenamiento', 'validación'], loc='upper left')
plt.show()

plt.figure(figsize=(15,8)) 
if (tipoProblemaAplicar == "C") or (tipoProblemaAplicar == "T"):
  plt.plot(history.history['accuracy'])
  if (tipoProblemaAplicar != "T"):
    plt.plot(history.history['val_accuracy'])
  plt.title('Gráfico de la Exactitud del Entrenamiento')
elif (tipoProblemaAplicar == "E"):
  plt.plot(history.history['root_mean_squared_error'])
  plt.plot(history.history['val_root_mean_squared_error'])
  plt.title('Gráfico de la Distancia Media Cuadrática Mínima del Entrenamiento')
  
plt.ylabel('')
plt.xlabel('epoch')
plt.legend(['entrenamiento', 'validación'], loc='upper left')
plt.show()

In [ ]:
#@title Evaluar red entrenada con datos de entrenamiento

mostrar_detalle_entrenamiento = False #@param {type:"boolean"}

# clase especial para generar texto
class TextGenerator():

  def __init__(self, model, text_encoder, tagBegin, tagEnd, sepTokens=" "):
      self.model = model
      self.textEncoder = text_encoder
      self.y_vocab = np.array(text_encoder.get_vocabulary())
      self.tagBegin = tagBegin
      self.tagEnd = tagEnd
      self.sepTokens = sepTokens
      self.cantModelInputs = self.determinarCantModelInputs()

  def determinarCantModelInputs(self):
      cantInputs = 0
      for layer in self.model.layers:
        if isinstance(layer, keras.layers.InputLayer):
          cantInputs = cantInputs + 1
      return cantInputs

  def samplePreds(self, preds, temperature):
      # función que se usa para generar mayor diversidad al generar texto
      # helper function to sample an index from a probability array
      preds = np.asarray(preds).astype("float64")
      if temperature > 0.0:
        preds = preds / temperature
      ##preds = np.log(preds) / temperature
      ##preds = np.nan_to_num(preds)
      exp_preds = np.exp(preds)
      preds = exp_preds / np.sum(exp_preds)
      probas = np.random.multinomial(1, preds, 1)
      return np.argmax(probas)

  def generar(self, datos=None, startText="", diversity=0.5):
      # prepara datos
      if (datos is None) or (len(datos)==0):
        input_x = None
        if self.cantModelInputs == 2:
          raise(Exception("No se ha indicado los datos complementarios!"))          
          return None
      else:
        input_x = np.array( [ datos ] )      
      # prepara texto inicio
      input_text = self.tagBegin 
      pos_sgte_input_text = 1
      if startText != "":
        input_text = input_text + self.sepTokens + startText
        y_texto = startText
      else:
        y_texto = ""
      # codifica texto de inicio      
      input_textCodif = np.array( [ self.textEncoder(input_text) ] )
      # determina siguiente posición a usar
      if startText != "":
        for i in range(len(input_textCodif[0])):
          if input_textCodif[0][i] == 0:
            pos_sgte_input_text = i
            break
      # chequea temperatura
      if diversity <= 0.0:
        diversity = 0.001
      for _ in range(100):
        # ejecuta modelo
        if self.cantModelInputs == 2:
          y = self.model.predict( [ input_x,  input_textCodif ], verbose=0 )
        else:
          y = self.model.predict( input_textCodif, verbose=0 )
        # determina salida
        id = self.samplePreds(y[0], diversity)
        ##id = int( np.argmax(y[0], axis=0) )    
        res = self.y_vocab[id]        
        # contrala si tiene que termina
        if (id == 0) or (res == self.tagEnd):
            break        
        # concatena texto de salida sólo si es caracter válido
        if checkValidCharacter(res):
          y_texto = y_texto + self.sepTokens + res
        else:
          print("-- se descarta caracter inválido: (", ord(res), ")")
          print("\t\t\t", res)
        # agrega salida para siguiente ejecución del modelo       
        input_textCodif[0][pos_sgte_input_text] = id
        pos_sgte_input_text = pos_sgte_input_text + 1    
      return y_texto


def probarModelo_GenTextoEjemplos(gt, x, y, mostrarTodos=False):    
  if mostrarTodos:
    # muestra varios (no todos porque sino tarda demasiado)
    cantTotal = len(y)    
    tomalAlAzar = False
    posEj = 0
    print("    ( procesando todos los "+str(cantTotal)+" ejemplos )")
  else:
    # prueba con <cantMostrar> ejemplos
    cantTotal = 10
    tomalAlAzar = True
    print("    ( procesando "+str(cantTotal)+" ejemplos al azar )")
  if (x is None) or (len(x)==0):
    x = None
    input_datos = None
  auxStartText = ""
  for i in range(cantTotal):
    if tomalAlAzar:
      # toma al azar
      posEj = random.randint(1, len(y))-1
    else:
      # toma secuencial
      posEj = posEj + 1
    print("\n", i+1, ") Ejemplo ", posEj)
    if not(x is None):
      # ejecuta con datos
      input_datos = x[posEj]
      print("* Datos: ", input_datos )
    else:
      # ejecuta sin datos y pasando la primera palabra
      auxStartText = separarTexto(y[posEj], separadorTokens)[0]    
    print("+ Texto Original: ", y[posEj] ) 
    if auxStartText != "":
      print("> se ingresa: ", auxStartText)
    # genera texto   
    for diversityPreds in [0.1, 0.5, 1.0, 1.5]:
      textoRes = gt.generar(datos=input_datos, startText=auxStartText, diversity=diversityPreds)
      print("- Texto Generado (diversity="+str(+diversityPreds)+"): ", textoRes )  
    print("--------------------------------------------------------------")


# función auxiliara para que no ejecute UI cada vez
def hacerNada():
  return

def prepararUI_GenTextoIngresado():
    global probar_startText, probar_diversity

    print("\n")

    # auxiliar para que muestre bien la descripción
    style_3D = {'description_width': 'initial'}

    # prepara campo para ingresar nombre clases (toma por defecto de config)
    probar_startText = widgets.Text(
        value='',
        placeholder='',
        description='Ingrese texto inicial:',
        style=style_3D,
        disabled=False
    )

    probar_diversity = widgets.FloatSlider(
        value=0.5,
        min=0.1,
        max=2.0,
        step=0.1,
        description='Diversity Temperature:',
        style=style_3D,
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
    )

    # prepara botón y grilla con objetos
    btnProbar = widgets.Button(
        description='Probar'
    )

    probarGentText_ui = widgets.GridBox(
          children=[probar_startText, probar_diversity, btnProbar],
          layout=Layout(width='100%')  )
    btnProbar.on_click(on_buttonProbar_clicked)

    # clear_output()      
    out_probarGenText = widgets.interactive_output(hacerNada, {})  
    display(probarGentText_ui)
    print("\n")

def on_buttonProbar_clicked(b): 
  auxStartText = probar_startText.value
  divTemp = probar_diversity.value
  # inicializa y usa generador de texto  
  textoRes = gt.generar(datos=None, startText=auxStartText, diversity=divTemp)    
  print("> se ingresa: ", auxStartText)
  print("- Texto Generado: ", textoRes )  
  print("--------------------------------------------------------------")

# función auxiliar para el cálculo de error
def calcErrores(pred, real, mostrarDetalle=False):
  arAbs = []
  arRel = []
  
  if mostrarDetalle:
    print("\n")
    print("\t Real \t\t\t Estimado \t\t Error Absoluto \t Error Relativo")

  for pV, r in zip(pred, real):
    # toma el valor estimado/predecido
    p = pV[0]
    # controla que sean números
    if not(math.isnan(r) or math.isnan(p)):
      # hace los cálculos
      eAbs = abs(r - p)
      if r != 0:
        eRel = (eAbs / r)*100.0
      else:
        eRel = (eAbs / 0.00001)*100.0
      arAbs.append(eAbs)
      arRel.append(eRel)
    
      if mostrarDetalle:
        print("\t{:>8.2f} \t\t {:>8.2f} \t\t {:>8.2f} \t\t {:>8.2f}%".format(r, p, eAbs, eRel))

  return arAbs, arRel

def generarGrafico(ar, tit, b=10, c=None):
     # genera gráfico de los errores
    fig = plt.figure(figsize=(15,5)) 
 #   ax = fig.add_axes( [0, 0, 0.8, 0.8] )
 #   ax.boxplot( [arAbs, arRel] )
 #   ax.set_xticklabels( ["Error Absoluto", "Error Relativo"] )
#    plt.legend(["Error Absoluto", "Error Relativo"], loc='best')
    plt.hist( ar, bins=b, color=c )
    plt.grid(color='lightgrey', which='both', axis='both', linestyle='solid', linewidth=0.3)
    plt.title("Distribución de "+ tit)
    plt.show()


# función auxiliar para probar el modelo entrenado en detalle
def probarModelo_Estimacion(x, y, detalle=False):

    # procesa las imágenes de prueba con el modelo 
    estimVals = model.predict(x, verbose=0)

    # llama a la función
    arAbs, arRel = calcErrores(estimVals, y, detalle)

    # muestra métricas
    print("\n")
    print("\n Error Absoluto: ")
    print("            Mínimo: {:.5f} ".format(np.min(arAbs)) )
    print("            Promedio: {:.5f} ± {:.5f}".format(np.mean(arAbs), np.std(arAbs)) )
    print("            Máximo: {:.5f} ".format(np.max(arAbs)) )
    generarGrafico(arAbs, "Error Absoluto", 20, "red")
    
    print("\n Error Relativo: ")
    print("            Mínimo: {:.2f}% ".format(np.min(arRel)) )
    print("            Promedio: {:.2f} ± {:.2f}".format(np.mean(arRel), np.std(arRel)) )
    print("            Máximo: {:.2f}% ".format(np.max(arRel)) )
    generarGrafico(arRel, "Error Relativo", 10, "magenta")


# función auxiliar para probar el modelo entrenado en detalle
def probarModelo_Clasificacion(x, y, clases_map, mostrarDetalle=False):

    # procesa las imágenes de prueba con el modelo 
    predClass = model.predict(x, verbose=0)

    # muestra los resultados con las imágenes 
    umbralClas = 0.5
    classPreds = []
    classReal = []
    for i in range(len(x)):

        # prepara salida
        clReal = clases_map[ int(y[i]) ] 

        # determina la clase predecida
        if tipo_output_softMax:
            ## determina clase predecida de acuerdo a la que tiene mayor valor
            idclPred = int( np.argmax(predClass[i], axis=0) )
            idclPredRnd = idclPred
        else:
            ## determina clase predecida de acuerdo al umbral de clasificación
            idclPred = predClass[i][0]       
            idclPredRnd = int(idclPred)
            if (idclPred - idclPredRnd)>0.5 and (idclPredRnd+1)<len(clases_map):
                    idclPredRnd = idclPredRnd + 1

        if idclPredRnd<0 or idclPredRnd>=len(clases_map):
            clPred = "CLASE " + str(idclPredRnd) + " INVÁLIDA"
        else:      
            clPred = clases_map[ idclPredRnd ]

        classReal.append( clReal )
        classPreds.append( clPred )

        strTitulo = 'Real: ' + str(clReal) + ' / Modelo(RNA): ' 
        strTitulo = strTitulo + str(clPred) + ' (' + str( idclPred ) +')'   
        strTitulo = strTitulo + ": " + ("ok" if (clPred==clReal) else "error!")

        # muestra comparación con la imagen
        if mostrarDetalle:
          print(strTitulo)

    # muestra reporte de clasificación
    print("\n Reporte de Clasificación: ")
    print(classification_report(classReal, classPreds))

    # muestra matriz de confusion
    print('\nMatriz de Confusión ( real / modelo ): ')
    cm = confusion_matrix(classReal, classPreds, labels=clases_map)
    cmtx = pd.DataFrame(
        cm, 
        index=['r:{:}'.format(x) for x in clases_map], 
        columns=['m:{:}'.format(x) for x in clases_map]
      )
    # agrega para poder mostrar la matrix de confusión completa
    pd.options.display.max_rows = 100
    pd.options.display.max_columns = 100
    cmtx.sort_index(axis=0, inplace=True)
    cmtx.sort_index(axis=1, inplace=True)    
    print(cmtx)
    print("\n")

    # gráfico de comparación
    plt.title('Gráfico de Confusión: ')
    plt.xlabel('Real')
    plt.ylabel('Modelo')
    plt.scatter(classReal, classPreds)

# prueba con los datos de entrenamiento
print("*** Resultados con datos de Entrenamiento: ")
if (tipoProblemaAplicar == "T"):
  # inicializa y usa generador de texto
  gt = TextGenerator(model, y_encoderLay, tag_gentext_begin, tag_gentext_end, separadorTokens)  
  # prueba gen texto  
  probarModelo_GenTextoEjemplos(gt, x_train, y_train, mostrar_detalle_entrenamiento)
  # prepara interface para ingresar otro texto
  # (se usa en la de prueba)
  ##prepararUI_GenTextoIngresado()
elif (tipoProblemaAplicar == "C"):
  # prueba clasificación
  probarModelo_Clasificacion(x_train, y_train, CLASES, mostrar_detalle_entrenamiento)
else:
  # prueba estimación
  probarModelo_Estimacion(x_train, y_train, mostrar_detalle_entrenamiento)


In [ ]:
#@title Evaluar red entrenada con datos de prueba
mostrar_detalle_prueba = False #@param {type:"boolean"}

if (tipoProblemaAplicar != "T"):
  # evalua al modelo entrenado
  resEval = model.evaluate(x_test, (y_testEnc if tipo_output_softMax else y_test), verbose=0)
  print("\n>Evaluación del Modelo: ")
  print("    - Error: ", round(resEval[0],3))
  if (tipoProblemaAplicar == "C"):
    print("    - Exactitud: ", round(resEval[1]*100,2))
  else:
    print("    - Distancia Media Cuadrática Mínima: ", round(resEval[1],3))
  
# prueba con los datos de prueba
print("\n\n*** Resultados con datos de Prueba: ")
if (tipoProblemaAplicar == "T"):
  if esArchivoDatos:
    # prueba gen texto con ejemplos de datos
    probarModelo_GenTextoEjemplos(gt, x_test, y_test, mostrar_detalle_prueba)
  else:
    # para archivo de textos
    # prepara interface para ingresar otro texto
    prepararUI_GenTextoIngresado()
elif (tipoProblemaAplicar == "C"):
  # prueba clasificación
  probarModelo_Clasificacion(x_test, y_test, CLASES, mostrar_detalle_prueba)
else:
  # prueba estimación
  probarModelo_Estimacion(x_test, y_test, mostrar_detalle_prueba)
